In [1]:
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [2]:
digits = load_digits()  #64
X = digits.data
y = digits.target
y = LabelBinarizer().fit_transform(y)  #transform y as one-hot
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.3)
#X_train = tf.cast(X_train, tf.float32)
#y_train = tf.cast(y_train, tf.float32)

In [3]:
X_train.shape

(1257, 64)

In [4]:
X_train.dtype

dtype('float64')

In [5]:
def add_layer(inputs, in_size, out_size, layer_name, activation_func=None):
    Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='weight')
    biases = tf.Variable(tf.zeros([1,out_size]) + 0.1, name='biases')
    Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
    #drpout
    Wx_plus_b = tf.nn.dropout(Wx_plus_b, keep_prob)
    if activation_func is None:
        output = Wx_plus_b
    else:
        output = activation_func(Wx_plus_b,)
    tf.summary.histogram(layer_name+'/outputs', output)
    return output

In [6]:
keep_prob = tf.placeholder(tf.float32) #dropout定义
xs = tf.placeholder(tf.float32,[None,64]) #8*8
ys = tf.placeholder(tf.float32,[None,10]) # range(10)

In [7]:
#add hidden layer
l1 = add_layer(xs,64,50,'l1',activation_func=tf.nn.tanh) #tanh 
#add output layer
prediction = add_layer(l1, 50, 10, 'l2', activation_func=tf.nn.softmax)  #output = 10, 0-9

In [8]:
#loss == the error between prediction and real data
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction), reduction_indices=[1])) #loss  -> tf.nn.softmax
tf.summary.scalar('loss',cross_entropy)

<tf.Tensor 'loss:0' shape=() dtype=string>

In [9]:
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(cross_entropy) #learning_rate = 0.05

In [10]:
sess = tf.Session()
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter("logs/train", sess.graph)
test_writer = tf.summary.FileWriter("logs/test", sess.graph)
sess.run(tf.global_variables_initializer())

In [11]:
for i in range(500):
    sess.run(train_step, feed_dict={xs:X_train, ys:y_train, keep_prob:0.5})  #keep_prob value = 1 - x%
    if i%50==0:
        train_result = sess.run(merged, feed_dict={xs:X_train,ys:y_train, keep_prob:1})
        test_result = sess.run(merged, feed_dict={xs:X_test,ys:y_test, keep_prob:1})
        train_writer.add_summary(train_result, i)
        test_writer.add_summary(test_result, i)